In [1]:
# This notebook is an implementation of an optical CNOT operation using PBSs. Based on: https://doi.org/10.1103/PhysRevA.64.062311
import qutip as qt
import numpy as np
from quimb.tensor.tensor_arbgeom import tensor_network_apply_op_vec
from matplotlib import pyplot as plt

from trajectree.optical_quant_info import *
from trajectree.fock_optics.outputs import read_quantum_state

/home/asingal/home/.venv/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/asingal/home/.venv/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
N = 2
mean_photon_num = 0.05
error_tolerance = 1e-10

In [3]:
# Defining the control and target inputs here
a_dag = qt.create(N).full()
I = qt.identity(N).full()

a_H = np.kron(a_dag, I)
a_V = np.kron(I, a_dag)

# Testing the case with two MPSs for control and target
vacuum = create_vacuum_state(num_modes=2, N=N)

# control_operator = 1/np.sqrt(3) * a_H - np.sqrt(2/3) * a_V
control_operator = a_V
control_MPO = create_MPO(0, 1, 2, control_operator, N, "control")
psi_control = tensor_network_apply_op_vec(control_MPO, vacuum, compress=True, contract = True, cutoff = error_tolerance)

# Our target is in the H state 
target_operator = a_V
target_MPO = create_MPO(0, 1, 2, target_operator, N, "target")
psi_target = tensor_network_apply_op_vec(target_MPO, vacuum, compress=True, contract = True, cutoff = error_tolerance)


# Testing the case with only one MPS for both control and target
vacuum = create_vacuum_state(num_modes=8, N=N)

# control_operator = 1/np.sqrt(3) * a_H + np.sqrt(2/3) * a_V
control_operator = a_H
control_MPO = create_MPO(2, 3, 8, control_operator, N, "control")
psi_control = tensor_network_apply_op_vec(control_MPO, vacuum, compress=True, contract = True, cutoff = error_tolerance)

# Our target is in the H state 
target_operator = 1/np.sqrt(3) * a_H + np.sqrt(2/3) * a_V
target_MPO = create_MPO(6, 7, 8, target_operator, N, "target")
psi_combined = tensor_network_apply_op_vec(target_MPO, psi_control, compress=True, contract = True, cutoff = error_tolerance)


# Testing the case with only one MPS for both control and target
vacuum = create_vacuum_state(num_modes=4, N=N)

# control_operator = 1/np.sqrt(3) * a_H + np.sqrt(2/3) * a_V
control_operator = a_H
control_MPO = create_MPO(0, 1, 4, control_operator, N, "control")
psi_control = tensor_network_apply_op_vec(control_MPO, vacuum, compress=True, contract = True, cutoff = error_tolerance)

# Our target is in the H state 
target_operator = a_H
target_MPO = create_MPO(2, 3, 4, target_operator, N, "target")
psi = tensor_network_apply_op_vec(target_MPO, psi_control, compress=True, contract = True, cutoff = error_tolerance)

# psi.draw()
# psi

In [4]:

# # First, we implement a beamsplitter. We found this beamsplitter configuration using trial and error to match the Hadamard transformation.
# unitary_H = generalized_mode_mixer(np.pi/2, np.pi/2, -np.pi/2, 2*np.pi, N)
# # Next, we implement the -pi/2 (single mode) phase shifters on the |V> mode, before and after the beamsplitter. 
# unitary_H = np.kron(single_mode_phase(-np.pi, N), np.eye(N)) @ unitary_H @ np.kron(single_mode_phase(-np.pi, N), np.eye(N))

# print(np.round(unitary_H, 4))

read_quantum_state(psi, N, num_states=2)

# Testing functions. 
psi = H(psi, sites=(0,1), N=N, error_tolerance=1e-10)

read_quantum_state(psi, N, num_states=2)

psi = CNOT((0,1), (2,3), psi, None, N, mean_photon_num, error_tolerance)

read_quantum_state(psi, N, num_states=2)

Corresponding Basis terms:
0H1V_B 0H1V_A - 10 - [1.-0.j]
Corresponding Basis terms:
1H0V_B 0H1V_A - 6 - [0.70710678+0.j]
0H1V_B 0H1V_A - 10 - [0.70710678+0.j]
Corresponding Basis terms:
0H0V_B 0H0V_A - 0 - [0.24937552-0.j]
1H0V_B 1H0V_A - 5 - [0.01247918+0.j]
0H1V_B 0H1V_A - 10 - [-0.01247918-0.j]
1H1V_B 1H1V_A - 15 - [-0.00062448-0.j]


In [5]:
vacuum = create_vacuum_state(num_modes=8, N=N)
bell_state, _ = light_source(vacuum, N, mean_photon_num, 8, error_tolerance, compress=True, contract=True)

ancilla = create_vacuum_state(num_modes=2, N=N)
ancillary_initializer = a_H
ancillary_MPO = create_MPO(0, 1, 2, ancillary_initializer, N, "ancilla")
ancilla = tensor_network_apply_op_vec(ancillary_MPO, ancilla, compress=True, contract = True, cutoff = error_tolerance)

psi = extend_MPS(ancilla, bell_state)
psi = extend_MPS(psi, ancilla)

# read_quantum_state(psi, N, num_states=4)

psi = H(psi, sites=(0,1), N=N, error_tolerance=1e-10)
psi = H(psi, sites=(6,7), N=N, error_tolerance=1e-10)

# read_quantum_state(psi, N, num_states=4)

psi = CNOT((0,1), (2,3), psi, None, N, mean_photon_num, error_tolerance)
psi = CNOT((6 ,7), (4,5), psi, None, N, mean_photon_num, error_tolerance)

# DEPOLARIZING NOISE COMES HERE

psi = H(psi, sites=(0,1), N=N, error_tolerance=1e-10)
psi = H(psi, sites=(6,7), N=N, error_tolerance=1e-10)

# read_quantum_state(psi, N, num_states=4)

psi = CNOT((0,1), (2,3), psi, None, N, mean_photon_num, error_tolerance)
psi = CNOT((6 ,7), (4,5), psi, None, N, mean_photon_num, error_tolerance)

read_quantum_state(psi, N, num_states=4, precision = 7)

Corresponding Basis terms:
0H0V_D 0H0V_C 0H0V_B 0H0V_A - 0 - [-1.13e-05+0.j]
0H0V_D 0H0V_C 1H0V_B 1H0V_A - 5 - [-4.e-07+0.j]
0H0V_D 0H0V_C 0H1V_B 1H0V_A - 9 - [-4.e-07+0.j]
0H0V_D 1H0V_C 1H0V_B 0H0V_A - 20 - [3.e-07-0.j]
0H0V_D 1H0V_C 0H1V_B 0H0V_A - 24 - [-3.e-07-0.j]
0H0V_D 0H1V_C 1H0V_B 0H0V_A - 36 - [-3.e-07-0.j]
0H0V_D 0H1V_C 0H1V_B 0H0V_A - 40 - [3.e-07+0.j]
1H0V_D 1H0V_C 0H0V_B 0H0V_A - 80 - [-4.e-07+0.j]
1H0V_D 0H1V_C 0H0V_B 0H0V_A - 96 - [-4.e-07-0.j]
0H1V_D 0H0V_C 0H0V_B 0H1V_A - 130 - [6.e-07-0.j]
